In [1]:
import pandas as pd
from models.class_calm_water_resistance_estimatoin import *
from inference.radded_predictor import predict_r_added_from_dir
from inference.compute_daily_fuel import *

In [6]:
# --------------------------------------------------
# 1. Load validation data used for inference
# --------------------------------------------------
df = pd.read_csv('validation_data_55.csv')

# Compute calm-water resistance (unit: kN) for each record
# Cal_R_calm is a user-defined function
df['R_calm'] = df.apply(Cal_R_calm, axis=1)  # kN

# --------------------------------------------------
# 2. Define input feature columns for R_added prediction
# --------------------------------------------------
feature_cols = [
    'SOG', 'heading', 'draught',
    'wind_val', 'wind_direction',
    'wave_val', 'wave_direction',
    'stream_val', 'stream_direction'
]

# --------------------------------------------------
# 3. Predict added resistance (R_added) using trained models
# --------------------------------------------------
# This function loads all required models from the given directory
# and outputs predicted R_added for each record
r_added, data_with_radded, scaler = predict_r_added_from_dir(
    df=df,
    model_dir="Modellib",
    feature_cols=feature_cols,
    scaler_name="scaler_inference.pkl",
    device="cpu"
)

# --------------------------------------------------
# 4. Compute hourly fuel consumption and aggregate to daily fuel / CO2
# --------------------------------------------------
# emission_factor = 3.15 corresponds to ton CO2 per ton fuel
df_hourly, df_daily = compute_daily_fuel(
    data_with_radded,
    time_col="postime",
    emission_factor=3.15
)

# --------------------------------------------------
# 5. Load true (MRV-based) daily fuel / emission data
# --------------------------------------------------
df_true = pd.read_csv('MRV_true.csv')

# Ensure date columns are in datetime format for proper merging
df_daily["date"] = pd.to_datetime(df_daily["date"])
df_true["date"] = pd.to_datetime(df_true["date"])

# Select only necessary columns from prediction results
df_daily_sub = df_daily[["Vessel ID", "date", "emission_predict_day"]]

# --------------------------------------------------
# 6. Merge true and predicted daily CO2 emissions
# --------------------------------------------------
# The merge is performed on Vessel ID and date
# Left join is used to preserve all true records
df_merged = df_true.merge(
    df_daily_sub,
    on=["Vessel ID", "date"],
    how="left"
)

df_merged

/slurm/home/yrd/luolab/zhaoliang/.conda/envs/py310/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,Vessel ID,date,fuel_true_day,emission_true_day,emission_predict_day
0,1,2021-09-23,71.43,225.0045,216.386105
1,1,2021-09-29,64.52,203.2380,210.359621
2,1,2021-10-02,62.02,195.3630,192.851040
3,1,2022-05-04,68.61,216.1215,210.895311
4,1,2022-07-10,52.42,165.1230,163.911538
5,1,2023-03-08,61.59,194.0085,185.930617
6,1,2023-06-06,60.67,191.1105,190.003571
7,1,2024-02-24,48.96,154.2240,164.527325
8,1,2024-03-08,63.94,201.4110,202.149131
9,2,2021-04-13,52.21,164.4615,167.098672
